fact orders


In [0]:
df=spark.sql("select * from databricks_ete.silver.orders_silver")
display(df)

In [0]:
df1= spark.sql("select * from databricks_ete.gold.dimproducts")
display(df)


In [0]:
df_dimcus=spark.sql("select dimkeys , customer_id as dim_cus_id from databricks_ete.gold.dimcustomers")
df_dimprod=spark.sql("select product_id as dimprodkeys , product_id as dim_prod_id from databricks_ete.gold.dimproducts")


In [0]:
df.display()

In [0]:
df_fact = df \
    .join(df_dimcus, df['customer_id'] == df_dimcus['dim_cus_id'], "left") \
    .join(df_dimprod, df['product_id'] == df_dimprod['dim_prod_id'], "left")

df_fact_new=df_fact.drop("dim_cus_id", "dim_prod_id","customer_id","product_id")

In [0]:
df_fact_new.display()

upsert on fact


In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_ete.gold.FactOrders"):
    
    dlt_obj = DeltaTable.forName(spark, "databricks_ete.gold.FactOrders")

    dlt_obj.alias("trg").merge(df_fact_new.alias("src"), "trg.order_id = src.order_id AND trg.dimkeys = src.dimkeys AND trg.dimprodkeys = src.dimprodkeys")\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()

else:
    df_fact_new.write.format("delta")\
            .option("path","abfss://gold@eteproject.dfs.core.windows.net/FactOrders")\
            .saveAsTable("databricks_ete.gold.FactOrders")


In [0]:
%sql 
select * from databricks_ete.gold.factorders